In [ ]:
import datetime as dt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tushare as ts
import seaborn as sns

# 读入期权行情数据
option_price = pd.read_csv('data/options.csv', parse_dates=[2])  # 读取并将特定列解析为时间类型
option_price

# 将相同到期时间和相同行权价格合约标记相同的group_mark，为合成straddle做准备
option_price['group_mark'] = option_price.ticker.map(lambda s: s[-10:-6] + s[-5:])
option_price.head()

# 定义apply方法的回调函数，将相同过期月份和行权价格的认沽和认购合约的收盘价相加组成straddle组合的收盘价，并计算straddle组合的日收益率
def get_straddle_data(df):
    straddle_price = df[['ticker', 'tradeDate', 'closePrice']].set_index(['tradeDate', 'ticker']).unstack().sum(axis=1)
    straddle_return = straddle_price.pct_change()
    df = pd.concat([straddle_price, straddle_return], axis=1).dropna()
    df.columns = ['close', 'daily_return']
    return df

# 根据group_mark对原始数据做分组处理，计算straddle组合的价格和日收益率
straddle_data = option_price.groupby('group_mark').apply(get_straddle_data)
straddle_data

# # 根据groupmark提取straddle组合的到期时间
straddle_data['expire'] = straddle_data.index.get_level_values(0).map(lambda s: s[:4])
straddle_data

#计算标的波动率
# 获取50ETF收盘价数据
etf = ts.get_k_data('510050', start='2014-01-01', end='2018-11-21')[['date', 'close']].set_index('date')
etf.index = pd.to_datetime(etf.index)
etf.head()

# 计算50ETF的收益率波动率
etf.sort_index(inplace=True)
etf['daily_return'] = etf.close.pct_change()
etf['volatility'] = etf.daily_return.rolling(60).std() #时间窗口至少要三十天以上标准差的计算结果才有意义，但太长的话近期数据的占比就过小无法很好地反映近期波动率的变化
etf.dropna(inplace=True)
etf.head()

# 计算波动率
vol_mean = etf.volatility.mean()
top10 = etf.volatility.quantile(0.1)

#波动率数据统计
# 波动率分布作图
etf.volatility.hist(figsize=(10,6), bins=100, alpha=0.5)
plt.axvline(top10, c='r', lw=1) #标记波动率10%分位点
plt.axvline(vol_mean, c='b', lw=1) #标记波动率均值
plt.show()

# 计算波动率的10%的移动分位点
etf['top10_vol']= etf.volatility.rolling(100).quantile(0.10)
etf.tail()

# 根据波动率计算开仓信号,开仓信号要求波动率出现连续10日收缩，收敛的标准是小于滚动的波动率的10%分位数
# 此单元最终的目的是统计出收敛信号连续出现的天数
# 此处使用的是统计连续值的常用思路
etf['top10_mark'] = np.where(etf.volatility < etf.top10_vol, 1, 0)                     # 收敛标记
etf['change_mark'] = np.where((etf.top10_mark - etf.top10_mark.shift()) == -1, 1, 0)   # 收敛结束标记
etf['group_mark'] = etf.change_mark.cumsum()                                           # 收敛分组标记(记录之前有几次收敛)
etf['open_signal'] = etf.groupby('group_mark').top10_mark.cumsum()                     # 计算每个收敛分组内收敛标记出现的次数
etf

# 检查持仓信号逻辑
etf.to_csv('data/check.csv')

# 多重索引中根据某一层索引值提取数据的方式
straddle_data.loc[(slice(None), "2018-11-14"), :].close.sort_values() #slice(None)表示该级index不进行筛选，这里选取二级index为2018-11-14的数据并根据close升序排列
straddle_data.loc[('160302400', slice(None)), :]
straddle_data.loc[('160302400', pd.to_datetime('2015/12/24')), :]

# 定义函数获取straddle组合在date日期的收益率
def get_p_return(date, straddle, straddle_df):
    try:
        p_return = straddle_df.at[(straddle, date), 'daily_return']
    except KeyError:   # 期权到期的情况，返回None
        return None
    return p_return

# 根据日期获得下个月的年月字符串组合
def get_next_month(date):
    month = date.month
    if month ==12:
        return f'{date.year+1}01'[2:]
    else:
        return f'{date.year}{month+1:02}'[2:]

# 定义函数，找到特定日期中晚于下个月到期且价格最低的straddle组合
def get_straddle(date, df_straddle):
    current_month = f'{date.year}{date.month:02}'[2:]
    next_month = get_next_month(date)
    trading_straddle = df_straddle.loc[(slice(None), date), :]   # 选取当日交易的所有straddle组合
    # 在所有交易的组合中，排除当月和下月到期的组合
    straddle_pool = trading_straddle[(trading_straddle.expire != current_month) & (trading_straddle.expire != next_month)]  
    return straddle_pool.close.idxmin()[0]                       # 返回备选组合中价格最低的组合的名称（日期和行权价组成的字符串）

# get_straddle函数使用示例
date = pd.to_datetime('2018-11-12')
get_straddle(date, straddle_data)

# 定义函数计算策略收益率
def get_portfolio_return(df_basic, df_straddle):
    """
    根据开仓信号和straddle组合的日收益率数据，计算策略的收益率。
    param df_basic: 包含开仓信号的DataFrame
    param df_straddle: straddle组合每日收益率数据，DataFrame类型
    return： 返回策略收益率，Series数据，索引结构与df_basic相同
    """
    holding_flag = False        # 持仓标记
    straddle = None             # 当前持有的straddle组合名称，字符串类型
    portfolio_returns = []      # 策略每日收益率
    trade_net_values = []       # 单笔交易自开仓起计算的净值，用于计算回撤空间和产生平仓信号
    trade_net_value = 1         # 单笔交易的初始净值
    max_net_value = 1           # 单笔交易的最大净值，用于计算单笔交易的最大回撤
    x_info = []                 # 其他想要记录的信息，非逻辑必须内容
    
    for idx, row in df_basic.iterrows(): #.interrows()每次迭代以(index, 非index数据)返回一行
        if holding_flag == False:           # 不持仓的情况下
            portfolio_returns.append(0)
            trade_net_values.append(1)
            x_info.append(None)
            if row.open_signal >= 10:       # 检查开仓信号
                holding_flag = True         # 修改持仓标记
                straddle = get_straddle(idx, df_straddle)             # 获取要持仓的straddle组合的名称
        else:   # 持仓的情况下
            p_return = get_p_return(idx, straddle, df_straddle)       # 获取组合持仓收益
            x_info.append(straddle)                                   # 记录持仓的straddle组合名称

            if p_return != None:                                      # straddle组合没有到期的情况下
                trade_net_value *= (p_return + 1)                     # 单笔交易的净值计算
                trade_net_values.append(trade_net_value)              # 记录单笔交易净值
                max_net_value = max(trade_net_value, max_net_value)   # 单笔交易最高净值
                portfolio_returns.append(p_return)                    # 记录策略日收益率
                trade_drawdown = trade_net_value/max_net_value        # 单笔交易回撤
            if (trade_drawdown < 0.8) or (p_return == None) :         # 出现平仓信号或合约到期(返回值为None)，重置所有临时变量
                holding_flag = False
                straddle = None
                trade_net_value = 1
                max_net_value = 1

    df_basic['straddle'] = pd.Series(x_info, index=df_basic.index)    
    df_basic['trade_net_value'] = pd.Series(trade_net_values, index=df_basic.index)

    return pd.Series(portfolio_returns, index=df_basic.index)

etf['p_return'] = get_portfolio_return(etf, straddle_data)
etf['net_value'] = (etf.p_return + 1).cumprod()
etf.net_value['2015':].plot(figsize=(12,6))
plt.show()
etf.to_csv('data/check.csv')